## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

## TODAY:

- Part A: We will divide our documents into CHUNKS
- Part B: We will encode our CHUNKS into VECTORS and put in Chroma
- Part C: We will visualize our vectors

### PART A: Divide our documents into chunks

In [1]:
import os
import glob
import tiktoken
import numpy as np
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [2]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "moonshotai/kimi-k2-instruct-0905"
db_name = "vector_db"
load_dotenv(override=True)
openai_api_key = os.getenv('kimi_k2_api_key')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")


OpenAI API Key exists and begins nvapi-JH


In [3]:
# How many characters in all the documents?

knowledge_base_path = "knowledge-base/**/*.md"
files = glob.glob(knowledge_base_path, recursive=True)
print(f"Found {len(files)} files in the knowledge base")

entire_knowledge_base = ""

for file_path in files:
    with open(file_path, 'r', encoding='utf-8') as f:
        entire_knowledge_base += f.read()
        entire_knowledge_base += "\n\n"

print(f"Total characters in knowledge base: {len(entire_knowledge_base):,}")

Found 30 files in the knowledge base
Total characters in knowledge base: 44,314


In [4]:
# Load in everything in the knowledgebase using LangChain's loaders

folders = glob.glob("knowledge-base/*")

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

print(f"Loaded {len(documents)} documents")

Loaded 30 documents


In [5]:
documents[0]

Document(metadata={'source': 'knowledge-base\\about\\bio.md', 'doc_type': 'about'}, page_content="# Professional Bio\n\nAyush Tyagi is a dedicated Computer Science Engineering student at JIMS, Greater Noida (IP University), on track to graduate in 2025. With a strong foundation built during his PCM schooling at Vivekanand School, Anand Vihar, he has channeled his analytical mindset into a passion for technology. \n\nAyush thrives on the creative process of front-end development, specializing in building visually stunning, user-friendly, and innovative web applications. He is a proactive learner, constantly seeking new challenges to expand his skill set and create digital experiences that are both functional and beautiful.\n\n## What Makes Me Unique\nMy uniqueness lies in the powerful combination of a creative eye and a logical, problem-solving mindset. I don't just write code; I craft experiences. I have a natural strength for designing interfaces that are not only aesthetically amazin

In [6]:
# Divide into chunks using the RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Divided into {len(chunks)} chunks")
print(f"First chunk:\n\n{chunks[7]}")

Divided into 57 chunks
First chunk:

page_content='# College Education

**Institution:** JIMS, Greater Noida  
**University:** IP University  
**Degree:** B.Tech in Computer Science and Engineering  
**Duration:** 2021 - 2025 (Expected)  
**Current CGPA:** 7.3

## Academic Focus
- Core computer science fundamentals
- Software engineering principles
- Data structures and algorithms
- Web technologies and development
- Artificial Intelligence and Machine Learning

## Key College Achievements
- Consistently maintaining good academic performance
- Active participation in technical projects and coding activities
- Developing practical skills through personal projects and internships
- Building a strong portfolio of web and game development projects

## Extracurricular Activities
- Technical project development
- Self-paced learning through online courses
- Participating in coding communities
- Building practical applications across multiple domains' metadata={'source': 'knowledge-base\\educ

In [8]:
chunks[10]

Document(metadata={'source': 'knowledge-base\\experience\\intensity_global.md', 'doc_type': 'experience'}, page_content='## Technologies Used\n- **Programming Languages:** Python\n- **ML Frameworks:** PyTorch, TensorFlow, Hugging Face Transformers\n- **AI Tools:** LangChain, LlamaIndex\n- **Vector Databases:** Pinecone, Chroma, FAISS\n- **Development Tools:** Git, Docker, Cloud Platforms\n\n## Key Achievements\n- Successfully deployed multiple fine-tuned LLM models for specific use cases\n- Improved model accuracy through advanced RAG system implementation\n- Contributed to production-level AI solutions\n- Enhanced prompt engineering strategies for better output quality\n\n## Skills Developed\n- Advanced LLM fine-tuning techniques\n- RAG system architecture and implementation\n- AI pipeline development and deployment\n- Cross-functional team collaboration\n- Performance evaluation and optimization')

### PART B: Make vectors and store in Chroma

In Week 3, you set up a Hugging Face account and got an HF_TOKEN

At this point, you might want to add it to your `.env` file and run `load_dotenv(override=True)`

(This actually shouldn't be required).

In [9]:
# Pick an embedding model

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
#embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 57 documents


In [15]:
# Let's investigate the vectors

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 413 vectors with 384 dimensions in the vector store


### Part C: Visualize!

In [10]:
print(vectorstore)
print(dir(vectorstore))



['_Chroma__ensure_collection', '_Chroma__query_collection', '_LANGCHAIN_DEFAULT_COLLECTION_NAME', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_asimilarity_search_with_relevance_scores', '_chroma_collection', '_client', '_collection', '_collection_configuration', '_collection_metadata', '_collection_name', '_cosine_relevance_score_fn', '_embedding_function', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_max_inner_product_relevance_score_fn', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_images', 'add_texts', 'adelete

In [ ]:
import chromadb
from chromadb.config import Settings

client = chromadb.PersistentClient(path="db")  # or your DB path
collection = client.get_collection(collection)  # name must match what you used earlier

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']

doc_types = [metadata['doc_type'] for metadata in metadatas]
colors = [['blue','green','red','orange'][['products','employees','contracts','company'].index(t)] for t in doc_types]


TypeError: argument 'name': 'Collection' object cannot be converted to 'PyString'

In [ ]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [ ]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig.show()